In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 设置设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 数据预处理
# 添加充足的图像增强方法
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),           # 随机裁剪，加上padding=4
    transforms.RandomHorizontalFlip(),              # 随机水平翻转
    transforms.RandomRotation(15),                  # 随机旋转，角度范围±15度
    transforms.ColorJitter(                         # 颜色抖动
        brightness=0.2,                             # 亮度调整范围
        contrast=0.2,                               # 对比度调整范围
        saturation=0.2,                             # 饱和度调整范围
        hue=0.1                                     # 色调调整范围
    ),
    transforms.RandomAffine(                        # 随机仿射变换
        degrees=0,                                  # 不额外旋转
        translate=(0.1, 0.1),                       # 平移范围
        scale=(0.9, 1.1),                           # 缩放范围
    ),
    transforms.ToTensor(),                          # 先转换为Tensor
    transforms.Normalize(                           # 然后标准化
        (0.4914, 0.4822, 0.4465), 
        (0.2023, 0.1994, 0.2010)
    ),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.2)), # 最后应用随机遮挡
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 加载CIFAR-10数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# CIFAR-10的类别
classes = ('飞机', '汽车', '鸟', '猫', '鹿', '狗', '青蛙', '马', '船', '卡车')

# 定义卷积神经网络
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, 
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.out_channel = 64
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, self.out_channel, kernel_size=7, stride=1, padding=3,
                               bias=False)    # 3表示输入特征图像的RGB通道数为3，即图片数据的输入通道为3
        self.bn1 = nn.BatchNorm2d(self.out_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# 创建ResNet-18模型
def ResNet18():
    return ResNet(ResidualBlock, [2, 2, 2, 2])

# 初始化模型
model = ResNet18().to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# 训练模型
def train(epoch):
    print(f'\n第 {epoch} 轮')
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 99:    # 每100个batch打印一次
            print(f'批次: {batch_idx+1}/{len(trainloader)} | 损失: {train_loss/(batch_idx+1):.3f} | 准确率: {100.*correct/total:.3f}%')
    
    return train_loss/len(trainloader), 100.*correct/total

# 测试模型
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    print(f'测试 | 损失: {test_loss/len(testloader):.3f} | 准确率: {100.*correct/total:.3f}%')
    
    return test_loss/len(testloader), 100.*correct/total

# 可视化一些测试图像
def visualize_sample():
    dataiter = iter(testloader)
    images, labels = next(dataiter)
    
    # 展示图像
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(np.transpose(images[i].numpy(), (1, 2, 0)))
        plt.title(classes[labels[i]])
        plt.axis('off')
    plt.show()
    
    # 使用模型预测
    images = images.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    
    print('预测结果:')
    for i in range(9):
        print(f'真实标签: {classes[labels[i]]}, 预测标签: {classes[predicted[i].item()]}')

# 训练和测试
train_losses = []
train_accs = []
test_losses = []
test_accs = []

epochs = 20  # 可以根据需要调整

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(epoch)
    test_loss, test_acc = test(epoch)
    scheduler.step()
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)

# 保存模型
torch.save(model.state_dict(), 'cifar10_resnet18.pth')
print('模型已保存')

# 绘制损失和准确率曲线
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='train_loss')
plt.plot(test_losses, label='test_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='train_accuracy')
plt.plot(test_accs, label='test_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()

# 可视化一些样本和预测结果
visualize_sample()

使用设备: cpu
Files already downloaded and verified
Files already downloaded and verified

第 1 轮
批次: 100/391 | 损失: 2.476 | 准确率: 19.703%
批次: 200/391 | 损失: 2.191 | 准确率: 24.172%


KeyboardInterrupt: 